### Load Wikipedia files into database

This files contains code to process the files from wikipedia, clean them and load them into the databse. 

In [ ]:
# import libraries
import pandas as pd
import os
from tempfile import mkstemp
from shutil import move
from os import fdopen, remove
import pickle
import numpy as np
import sqlalchemy as sql
import csv
from tqdm import tqdm_notebook
import wikipedia
from multiprocessing import Pool
import datetime

In [ ]:
DIR = "/home/justina/Desktop/dv/data_viz/server/wiki-data/2016-2018"
lst_data= os.listdir(DIR)

Helper functions to read teh file from the disk

In [ ]:
def replace(file_path):
    fh, abs_path = mkstemp()
    with fdopen(fh, 'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                head = line.split(';')[0:-2]
                tail= line.split(';')[-2:]
                tail = ' '.join(tail)
                head = ';'.join(head)
                final = head + ' ' + tail
                new_file.write(final)
                
    old = len(open(file_path).readlines())
    new = len(open(abs_path).readlines())
    
    print("Number of lines: ")
    print("{}, Old: {}, New: {}".format(file_path,old, new))
    
    if old == new:
        move(abs_path, file_path)
        
for file in lst_data:
    if 'pagecounts' in file:
        if 'spaces' not in file:
            replace(os.path.join(DIR, file))

For each data file, we have to sort them and get only the top 1000, so that we focus only on the most viewed articles.

In [6]:
def create_df(folder_of_files):
    # df of article name, monthly viewcount, day
    
    lst_data = os.listdir(folder_of_files)
    # lst_data=['pagecounts-2016-10-views-ge-5_cleaned']
    columns = ['article_name', 
               'monthly_viewcount', 
               'day']
    
    df = pd.DataFrame([])
    year_month = []
    for file in lst_data:
        if 'pagecounts' in file:
            next_df = pd.read_csv(os.path.join(folder_of_files, file), \
                                  sep=' ', \
                                  dtype={columns[0]:str, \
                                         columns[1]:int, columns[2]:int},\
                                  header=None, names=columns)
            
            lst_filename = file.split('-')
            yr = lst_filename[1]
            mth = lst_filename[2]
            next_df['year'] = yr
            next_df['month'] = mth
            year_month.append((int(yr), int(mth)))
            
            # sort them by most viewed
            next_df.sort_values(by='monthly_viewcount', ascending=False, inplace=True)
            print("Before filtering {}-{}: {}".format(yr,mth,next_df.year.size))
            
            # Filter unwanted data out
            mask = next_df.article_name.str.contains('Special:') | \
                    next_df.article_name.str.contains('Main_Page') | \
                    next_df.article_name.str.contains('Portal:') | \
                    next_df.article_name.str.contains('Wikipedia:') |\
                    next_df.article_name.str.contains('List of ') |\
                    next_df.article_name.str.contains('Special%:') |\
                    next_df.article_name.str.contains('User:') |\
                    next_df.article_name.str.contains('Help:') |\
                    next_df.article_name.str.contains('Category:') |\
                    next_df.article_name.str.contains('-') |\
                    next_df.article_name.str.contains('404.php') |\
                    next_df.article_name.str.contains('File:')
            
            next_df = next_df[~mask]
            print("After filtering: {}".format(next_df.year.size))
            
            # add them to our main data frame
            df = pd.concat([df, next_df.head(1000)], ignore_index = True)
    
    df = df.astype({'article_name': 'str', 'monthly_viewcount': 'int64', 
                    'day': 'int64', 'year': 'int64', 'month':'int64'})
    
    df['peak_date'] = pd.to_datetime(df[['year', 'month', 'day']])
    return df



Create a dataframe whichi contains all data we're interested in 

In [ ]:
df = create_df(os.path.join(DIR, 'final'))

In [ ]:
df.groupby(['year', 'month']).count()

In [ ]:
wikipedia.set_lang('en')
wikipedia.set_rate_limiting(rate_limit = True)

def get_article_data(x):
    row = x
    try:
        name = row['article_name']
        page = wikipedia.WikipediaPage(title=name)
        return name, page, row
        
    except:
        name = row['article_name']
        print(name)
        return name, None, row

def insert(lst, con, articles, article_name, \ 
           year, month, day, view_count, peak_date, \
           summary, page_id):
    
    lst.append({'title': article_name,
                   'year': year,
                   'month':month,
                   'day':day,
                   'view_count':view_count,
                   'peak_date': peak_date,
                   'summary': summary,
                   'page_id': page_id})
    
    if len(lst) > 50:
        print("Inserting...")
        con.execute(articles.insert(), lst)
        lst = []
    return lst

In [ ]:
fromdb_df = pd.read_csv('./2016-2018/top_1000_2016-2018/fromdb_data.csv', \
                        names = ['article_name', 'summary', 'page_id'])
fromdb_df.head()

In [ ]:
def get_final_data(filepath):    
    # df has title, year, month, day, viewcount, peak date
    fromdb_df = pd.read_csv('./2016-2018/top_1000_2016-2018/fromdb_data.csv', \
                            names = ['article_name', 'summary', 'page_id'], na_values = '')
    
    db_dict = {}
    for i, row in fromdb_df.iterrows():
        db_dict[row['article_name']] = (row['summary'], row['page_id'])
        
    article_names = set(fromdb_df.article_name.values)
    new_df = pd.DataFrame(columns = ['title', 'year', 'month', 'day', \
                                     'view_count', 'peak_date', 'summary', 'page_id'])

    df = pd.read_csv(filepath)
    exists = []
    
    for _, row in tqdm_notebook(df.iterrows(), total = len(df)):
        try:
            title = row['article_name']
            year = row['year']
            month = row['month']
            day = row['day']
            view_count =row['monthly_viewcount']
            peak_date = row['peak_date']

            if (title in db_dict):
                summary = db_dict[title][0]
                page_id = db_dict[title][1]
            else:
                _, page, _ = get_article_data(row)
                if page:
                    summary = page.summary
                    page_id = page.pageid
                    
                else:
                    summary = ""
                    page_id = -1
                db_dict[title] = (summary, page_id)
            
            new_df = new_df.append({'title': title,
                          'year': year,
                          'month': month,
                          'day': day,
                          'view_count': view_count,
                          'peak_date': peak_date,
                          'summary': summary,
                          'page_id': page_id}, ignore_index= True)
            
        except Exception as e: 
            # In case of failure we don't want the whole processing to stop
            print("Failed", row['article_name'])
            print(e)

    new_df.to_csv('./2016-2018/top_1000_2016-2018/final_data_0.csv')
    return new_df

In [ ]:
get_final_data('./2016-2018/top_1000_2016-2018/partial_view_data0.csv')

In [ ]:
df.to_csv('./wiki-data/2016-2018/top_1000_2016-2018/view_data.csv')
df_array = np.array_split(df,2)
for i, dframe in enumerate(df_array):
    dframe.to_csv('./wiki-data/2016-2018/top_1000_2016-2018/partial_view_data{}.csv'.format(i))

## Add may 2017 data to db again

In [ ]:
TOP1000_DIR = "/home/justina/Desktop/dv/data_viz/server/wiki-data/top_1000_per_month"

df1 = pd.DataFrame([])
for f in os.listdir(TOP1000_DIR):
    if 'counts' in f:
        df1 = pd.concat([df1, pd.read_csv(os.path.join(TOP1000_DIR, f))], ignore_index = True)
        
df1.year.size

In [ ]:
df1[(df1.month == 5) & (df1.year == 2017)].head(50)[['article_name', 'peak_date']]

In [ ]:
def add_entries(engine_path, df):
    engine = sql.create_engine(engine_path)
    metadata = sql.MetaData()
    articles = sql.Table('articles', metadata,
        sql.Column('title', sql.String),
        sql.Column('year', sql.Integer),
        sql.Column('month', sql.Integer),
        sql.Column('day', sql.Integer),
        sql.Column('view_count', sql.Integer), 
        sql.Column('peak_date', sql.Date),
        sql.Column('summary', sql.Text),
        sql.Column('page_id', sql.Integer)
    )
    metadata.create_all(engine)

    result = []
    errors = []
    wiki_df = pd.DataFrame(columns=['article_name', 'summary', 'page_id'])

    with engine.connect() as con:
        lst = list(df.iterrows())
        for _, row in lst:
            name = row['article_name']
            year = row['year']
            month = row['month']
            day = row['day']
            view_count = row['monthly_viewcount']
            peak_date = datetime.datetime.strptime(row['peak_date'], '%Y-%m-%d').date()

            if name not in wiki_df.article_name.values:
                name, page, r = get_article_data(row)

                if page:
                    wiki_df = wiki_df.append({'article_name': name, 'summary': page.summary, 'page_id': page.pageid}, ignore_index = True)
                else:
                    wiki_df = wiki_df.append({'article_name': name, 'summary': "", 'page_id': -1}, ignore_index = True)

                result = insert(result,con, articles, name, year, month, day, view_count, peak_date, wiki_df[wiki_df['article_name'] == name].summary.values[0], \
                                wiki_df[wiki_df['article_name'] == name].page_id.values[0])  

        con.execute(articles.insert(), result)
        return wiki_df

### Load file into database

In [ ]:
columns = ['title', 'year', 'month', 'day', 'view_count', 'peak_date', 'summary', 'page_id']

df_input0 = pd.read_csv('./2016-2018/top_1000_2016-2018/final_data_0.csv', names = columns, skiprows=1)
df_input1 = pd.read_csv('./2016-2018/top_1000_2016-2018/final_data_1.csv', names = columns, skiprows=1)
df_all = pd.concat([df_input0, df_input1], ignore_index=True)
df_all = df_all.reset_index()


In [ ]:
mask = (df_all.page_id ==-1)
df_all = df_all[~mask].iloc[:,1:]
print("Remove where page id is -1 ", len(df_all))

df_all = df_all[~df_all.title.str.contains("Template:")]
print("Remove articles with Template: ", len(df_all))

In [ ]:
df_input = df_all

In [ ]:
engine = sql.create_engine('sqlite:///kiru_db.db')
metadata = sql.MetaData()
articles = sql.Table('articles', metadata,
                sql.Column('title', sql.String),
                sql.Column('year', sql.Integer),
                sql.Column('month', sql.Integer),
                sql.Column('day', sql.Integer),
                sql.Column('view_count', sql.Integer),
                sql.Column('peak_date', sql.Date),
                sql.Column('summary', sql.Text),
                sql.Column('page_id', sql.Integer)
                )
metadata.create_all(engine)


with engine.connect() as con:

    for row in tqdm_notebook(df_input.iterrows(), total=len(df_input)):
        row = row[1]
        title = row['title']
        year = row['year']
        month = row['month']
        day = row['day']
        view_count =row['view_count']
        peak_date = row['peak_date']
        summary = row['summary']
        page_id = row['page_id']

        peak_date = datetime.datetime.strptime(peak_date, '%Y-%m-%d').date()
        con.execute(articles.insert(), {'title': title,
                                        'year': year,
                                        'month': month,
                                        'day': day,
                                        'view_count': view_count,
                                        'peak_date': peak_date,
                                        'summary': summary,
                                        'page_id': page_id})